In [1]:
import numpy as np
import open3d as o3d
from tqdm import tqdm_notebook
import os, sys, shutil
import matplotlib.pyplot as plt
import cv2

In [2]:
IMG_FOLDER_PATH = "/home/saby/Projects/ati/data/data/datasets/Carla/64beam-Data/pair_transform_dynseg_all/emptycities_preprocess/test/rs_img_40_512/"

In [3]:
def linear_transform(sample_img, A=0, B=255, a=-1, b=1):
    mul_fact = (b-a)/(B-A)
    sub_fact = A
    add_fact = a
    out_img = (sample_img-sub_fact)*mul_fact + add_fact
    return out_img

def to_polar_np(velo):
    if len(velo.shape) == 4:
        velo = velo.transpose(1, 2, 3, 0)

    if velo.shape[2] > 4:
        assert velo.shape[0] <= 4
        velo = velo.transpose(1, 2, 0, 3)
        switch=True
    else:
        switch=False
    
    # assumes r x n/r x (3,4) velo
    dist = np.sqrt(velo[:, :, 0] ** 2 + velo[:, :, 1] ** 2)
    # theta = np.arctan2(velo[:, 1], velo[:, 0])
    out = np.stack([dist, velo[:, :, 2]], axis=2)
    
    if switch:
        out = out.transpose(2, 0, 1, 3)

    if len(velo.shape) == 4: 
        out = out.transpose(3, 0, 1, 2)
    
    return out

def getint(name):
    try:
        return int(name.split('.')[0])
    except Exception as e:
        print("Error occured while trying to read {}".format(name))
    return None

In [4]:
img_list = []
for file in tqdm_notebook(sorted(os.listdir(IMG_FOLDER_PATH), key=getint)):
    img_arr = cv2.imread(os.path.join(IMG_FOLDER_PATH, file))
    temp_arr = np.concatenate([img_arr[:,:,2:3], img_arr[:,:,1:2], img_arr[:,:,0:1]], axis=-1)
    img_list.append(temp_arr)

/home/saby/anaconda3/envs/ati/lib/python3.6/site-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [5]:
img_list_arr = np.array(img_list)
img_list_arr.shape

(1274, 40, 512, 3)

In [6]:
img_transposed_arr = img_list_arr.transpose(0, 3, 1, 2)
img_transposed_arr.shape

(1274, 3, 40, 512)

In [7]:
img_orig_arr = linear_transform(img_transposed_arr)
img_orig_arr.shape

(1274, 3, 40, 512)

In [8]:
img_size = img_orig_arr.shape
npy_3d_arr = np.concatenate([img_orig_arr, np.zeros((img_size[0],1, img_size[2],img_size[3]))], axis=1)
npy_3d_arr.shape

(1274, 4, 40, 512)

In [9]:
npy_2d_arr = to_polar_np(npy_3d_arr.transpose(0,2,3,1)).transpose(0, 3, 1, 2)
npy_2d_arr.shape

(1274, 2, 40, 512)

In [10]:
np.save(os.path.join("/home/saby/Projects/ati/data/data/datasets/Carla/64beam-Data/pair_transform_dynseg_all/emptycities_preprocess/test/rs", "emptycities_testrs15.npy"), npy_2d_arr)